# 券商分點

這個券商分點可以繞過「圖片認證碼」請大家再三保密使用XD
不然哪天就失效了，

抓取券商分點，每檔股票總共可以分成三個步驟：
1. 開啟驗證碼網頁，得到驗證碼
2. 利用驗證碼來得到券商分點
3. 整理券商分點資訊

In [2]:
# ---------------
# 偽裝成瀏覽器
# ---------------
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:2.0.1) Gecko/2010010' \
    '1 Firefox/4.0.1',
    'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language':'en-us,en;q=0.5',
    'Accept-Charset':'ISO-8859-1,utf-8;q=0.7,*;q=0.7'}

# 1. 開啟驗證碼網頁，得到驗證碼
這邊的驗證碼不是「圖片驗證碼喔！」而是 `viewstate` 跟 `eventvalidation`

In [6]:
# ---------------
# 開啟有驗證碼的網頁
# ---------------
import requests
rs = requests.session()

def get_verify_code(rs):
    res = rs.get( 'http://bsr.twse.com.tw/bshtm/bsMenu.aspx', stream = True, verify = False, headers = headers, timeout=None )
    # ---------------
    # 從網頁中抓取一些關鍵字
    # ---------------
    import re
    
    # get view state
    viewstate = re.search( 'VIEWSTATE"\s+value=.*=', res.text )
    viewstate = viewstate.group()[18:]
    
    # get eventvalidation
    eventvalidation = re.search( 'EVENTVALIDATION"\s+value=.*\w', res.text )
    eventvalidation = eventvalidation.group( )[ 24: ]
    
    return viewstate, eventvalidation

viewstate, eventvalidation = get_verify_code(rs)

print('----')
print('viewstate', viewstate)
print('----')
print('eventvalidation',eventvalidation)
print('----')

----
viewstate /wEPDwUJOTAxMjkxMjg3D2QWAgIDD2QWBAIBD2QWAmYPZBYEZg9kFgJmDxAPFgYeB0NoZWNrZWRnHglGb3JlQ29sb3IKAB4EXyFTQgIEZGRkZAIBD2QWAgIBDw8WBB8BCgAfAgIEZGQCAw9kFgQCAw8PFgIeBFRleHRlZGQCCQ8PFgIeB1Zpc2libGVoZGQYAgUeX19Db250cm9sc1JlcXVpcmVQb3N0QmFja0tleV9fFgMFElJhZGlvQnV0dG9uX05vcm1hbAUQUmFkaW9CdXR0b25fRXhjZAUQUmFkaW9CdXR0b25fRXhjZAUPQ2FwdGNoYUNvbnRyb2wxDwUkMWQzY2RhYTYtNzE0YS00NjhiLTg4NzgtYzY0ZDE3ZjNiZDI2ZMpvSf1tcZC6GZQ5nfa8aPY=
----
eventvalidation /wEdAAYlzbq6Zytuh63xi01PqkaVfMmuxAJNAJcNkRRsVeJwwqKURZav/+YrVMqaWE2hvMxJwK4Ohf3nRgXHkFhQen1PRSciNdPYWmINCrip1wqw01PJCA0uL9aE2sjICZqv6GolupMMH5m3bSC97WVogYv9
----


# 2. 透過驗證碼爬取資料並存檔

In [7]:
# ---------------
# 爬取資料
# ---------------

def get_data(rs, stock_id, viewstate, eventvalidation):
    import time
    payload = {
    '__EVENTTARGET':'',
    '__EVENTARGUMENT':'',
    '__LASTFOCUS':'',
    '__VIEWSTATE' : viewstate,                      #encode_viewstate[:-1],
    '__EVENTVALIDATION' : eventvalidation,          #encode_eventvalidation[:-1],
    'RadioButton_Normal' : 'RadioButton_Normal',
    'TextBox_Stkno' : stock_id,
    'CaptchaControl1 ' : 'Z67YB',
    'btnOK' : '%E6%9F%A5%E8%A9%A2',
    }

    rs.post( "http://bsr.twse.com.tw/bshtm/bsMenu.aspx", data=payload, headers=headers)#, verify = False, stream = True )
    time.sleep(1)
    res = rs.get( 'http://bsr.twse.com.tw/bshtm/bsContent.aspx')#,verify = False ,stream = True, )
    return res

res = get_data(rs, '1101', viewstate, eventvalidation)
res.encoding='big5'
open('test.html', 'w', encoding='utf-8').write(res.text)

34127

# 3. 整理資料

In [8]:
import pandas as pd
from io import StringIO

def parse_data(text):
    lines = text.split('\n')
    lines = [l for l in lines if len(l.split(',')) == 11]
    df = pd.read_csv(StringIO('\n'.join(lines)))

    first_df = df[df.columns[:5]]
    second_df = df[df.columns[6:]]
    second_df.columns = second_df.columns.str.replace('.1', '')
    final_df = first_df.append(second_df).set_index('序號').sort_index().dropna()
    return final_df


df = parse_data(res.text)

# 完整下載範例（每天來按一下）

假如無法下載，可以參考
http://finlabcourse.imotor.com/viewthread.php?tid=322&extra=page%3D1&page=3
來做一些更動，就可以順利執行了～

In [ ]:
import time
import requests
from io import StringIO
import pandas as pd
import datetime

headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:2.0.1) Gecko/2010010' \
    '1 Firefox/4.0.1',
    'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language':'en-us,en;q=0.5',
    'Accept-Charset':'ISO-8859-1,utf-8;q=0.7,*;q=0.7'}

def get_data(rs, stock_id, viewstate, eventvalidation):
    import time
    payload = {
    '__EVENTTARGET':'',
    '__EVENTARGUMENT':'',
    '__LASTFOCUS':'',
    '__VIEWSTATE' : viewstate,                      #encode_viewstate[:-1],
    '__EVENTVALIDATION' : eventvalidation,          #encode_eventvalidation[:-1],
    'RadioButton_Normal' : 'RadioButton_Normal',
    'TextBox_Stkno' : stock_id,
    'CaptchaControl1 ' : 'Z67YB',
    'btnOK' : '%E6%9F%A5%E8%A9%A2',
    }

    rs.post( "http://bsr.twse.com.tw/bshtm/bsMenu.aspx", data=payload, headers=headers)#, verify = False, stream = True )
    time.sleep(1)
    res = rs.get( 'http://bsr.twse.com.tw/bshtm/bsContent.aspx')#,verify = False ,stream = True, )
    return res

def get_verify_code(rs):
    res = rs.get( 'http://bsr.twse.com.tw/bshtm/bsMenu.aspx', stream = True, verify = False, headers = headers, timeout=None )
    # ---------------
    # 從網頁中抓取一些關鍵字
    # ---------------
    import re
    
    # get view state
    viewstate = re.search( 'VIEWSTATE"\s+value=.*=', res.text )
    viewstate = viewstate.group()[18:]
    
    # get eventvalidation
    eventvalidation = re.search( 'EVENTVALIDATION"\s+value=.*\w', res.text )
    eventvalidation = eventvalidation.group( )[ 24: ]
    
    return viewstate, eventvalidation

def download_stock(stock_id, path):
    rs = requests.Session()
    #viewstate, eventvalidation = get_verify_code(rs)
    
    while True:
        try:
            viewstate, eventvalidation = get_verify_code(rs)
            break # 抓資料成功,進行下一步
        except Exception as e:
            print(e) # 印出無法連線原因
            print('無法拿到資料, 等 31 sec') # server好像會擋30sec
            time.sleep(31)
            continue # 抓資料失敗,重新抓資料
    
    time.sleep(2)
    res = get_data(rs, stock_id, viewstate, eventvalidation)
    res.encoding='big5'
    open(path, 'w', encoding='utf-8').writelines(res.text)
    
def stock_list():
    res = requests.get('http://dts.twse.com.tw/opendata/t187ap03_L.csv')
    res.encoding='big5'
    df = pd.read_csv(StringIO(res.text), index_col=['公司代號'])
    return df

import os

# build main dir
main_dir = os.path.join('data', 'broker_history')
if not os.path.isdir(main_dir):
    os.mkdir(main_dir)
    
# build date dir
now = datetime.datetime.now()
date_dir = os.path.join(main_dir, now.strftime('%Y%m%d'))
if not os.path.isdir(date_dir):
    os.mkdir(date_dir)

    
slist = stock_list()
for s in slist.index:
    stock_path = os.path.join(date_dir, str(s) + '.csv')
    print(stock_path)
    if not os.path.isfile(stock_path):
        print('downloading...')
        download_stock(str(s), stock_path)
        time.sleep(10)
    

# 券商總買賣

In [11]:
buy = df['買進股數'].astype(int).groupby(df['券商']).sum()
sell= df['賣出股數'].astype(int).groupby(df['券商']).sum()
total = buy - sell
total

券商
1020合　　庫        4000
1021合庫台中        1000
1022合庫台南        1000
1023合庫高雄           0
1024合庫嘉義       -7000
102C合庫自強        5000
1032土銀台南        2000
1040臺    銀    283000
1041臺銀鳳山      -13100
1042臺銀臺南        3000
1043臺銀民權        1000
1045臺銀新竹      -10000
104A臺銀臺中       20000
104C臺銀高雄        2650
104D臺銀金山       -8000
1116企銀屏東       -1000
111G企銀埔墘       -1000
1160日    盛    -21000
1161日盛忠孝        1000
1163日盛台中      -13500
1165日盛板橋        2000
1166日盛雙和        1000
1168日盛高雄        1000
1169日盛士林        4815
116A日盛木柵        1000
116C日盛八德       21000
116F日盛員林       -5000
116G日盛景美        5000
116I日盛南京       -2000
116L日盛大墩        1000
               ...  
989Z元大小港     -129000
989e元大中和       42000
989j元大樹林       -1000
989k元大古亭        2000
989m元大信義       -1000
9A00永豐金      -824000
9A8F永豐敦南      -20000
9A91永豐松山        8000
9A93永豐大稻       -4000
9A96永豐博愛        2000
9A98永豐大園         100
9A99永豐中壢        2000
9A9C永豐員林        3000
9A9G永豐天母        8000
9A9K永豐三重        -400
9A9L永豐台中        2000
9A9N永豐桃盛  

# 主力買賣超

In [13]:
# 買方top 15
total.nlargest(15)

券商
1440美林        806000
9200凱基        465000
1380台灣匯立      454000
1480美商高盛      444000
8960上海匯豐      422000
8440摩根大通      393000
9307華南大安      320000
1470台灣摩根      318000
1040臺    銀    283000
8560新    光    270000
9813元大八德      160000
5850統    一    150000
9695富邦民雄      140000
981N元大金門      112000
8840玉    山    100000
dtype: int64

In [162]:
# 賣方 top 15
total.nsmallest(15)

券商
8440摩根大通     -3093000
1590花旗環球     -1979000
8880國泰綜合     -1769000
9200凱基       -1497000
9800元大        -958000
9300華南永昌      -889000
9131群益民權      -355000
8580聯    邦    -340000
8840玉    山    -315000
779c國票敦北      -254000
6450永    全    -250000
1233彰銀台中      -240000
7001兆豐嘉義      -186000
5600永    興    -123000
9211凱基台南      -104000
dtype: int64

In [166]:
# 主力買賣超
total.nlargest(15).sum() + total.nsmallest(15).sum()

-559605

# 買賣家數差

In [164]:
# 買賣家數差
(total > 0).sum() - (total < 0).sum()

45